# POST Processing POD Reports - Pipeline Aligned & Updated

This notebook verifies the uniqueness of Penn holdings identified by the main processing pipeline. It reads from the standardized outputs in `pod-processing-outputs/` and performs final validation using the BorrowDirect API.

## Key Integration Points:
1. **Input**: Reads from pipeline outputs in order of preference:
   - `pod-processing-outputs/statistical_sample_with_borrowdir_results.parquet` (if BorrowDirect results exist)
   - `pod-processing-outputs/physical_books_no_hsp.parquet` (main filtered dataset)
   - `pod-processing-outputs/unique_penn_corrected.parquet` (basic unique Penn records)
2. **HSP Filtering**: Already applied in main pipeline (conditionally applied here if needed)
3. **BorrowDirect Results**: Leverages existing results or performs fresh API calls
4. **Output**: Saves confirmed unique records to `pod-processing-outputs/penn_unique_confirmed.xlsx`

## Workflow:
- Load data from main pipeline outputs with robust column handling
- Apply HSP filtering only if not already done in main pipeline  
- Use existing BorrowDirect results or fetch fresh data via API
- Perform Selenium-based holdings verification for final confirmation
- Export Penn-only holdings to Excel for manual review

The pipeline now correctly handles different output formats and leverages all enhancements from the main processing notebook.

In [ ]:
# Load data from main pipeline outputs - Updated and Robust
import pandas as pd
import numpy as np
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size

# Initialize Spark if needed
try:
    spark = SparkSession.getActiveSession()
    if spark is None:
        spark = SparkSession.builder \
            .appName("PostProcessing-Aligned") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
            .getOrCreate()
    print("✅ Spark session ready")
except:
    print("⚠️ Spark not available, using pandas for file reading")
    spark = None

# Replace the input_files list in post-processing.ipynb with:
input_files = [
    "pod-processing-outputs/statistical_sample_for_api_no_hsp.parquet",  # Sample from pod-processing
    "pod-processing-outputs/physical_books_no_533.parquet",  # Final filtered dataset (533 removed)
    "pod-processing-outputs/unique_penn.parquet",  # Basic unique Penn records
    "pod-processing-outputs/penn_overlap_analysis.parquet",  # Alternative analysis file
    "unique_penn_text.xlsx"  # Legacy Excel fallback
]

# Try to load from pipeline outputs
df = None
loaded_from = None

for input_file in input_files:
    if os.path.exists(input_file):
        try:
            print(f"📂 Attempting to load: {input_file}")
            if input_file.endswith('.parquet'):
                if spark:
                    df_spark = spark.read.parquet(input_file)
                    df = df_spark.toPandas()
                else:
                    df = pd.read_parquet(input_file)
                loaded_from = input_file
                print(f"✅ Loaded {len(df):,} records from {input_file}")
                break
            elif input_file.endswith('.xlsx'):
                df = pd.read_excel(input_file)
                loaded_from = input_file
                print(f"✅ Loaded {len(df):,} records from {input_file}")
                break
            elif input_file.endswith('.csv'):
                df = pd.read_csv(input_file)
                loaded_from = input_file
                print(f"✅ Loaded {len(df):,} records from {input_file}")
                break
        except Exception as e:
            print(f"❌ Failed to load {input_file}: {e}")
            continue

if df is None:
    raise FileNotFoundError("❌ No valid input files found. Please run the main pipeline first.")

print(f"\n🎯 Dataset loaded from: {loaded_from}")
print(f"📊 Shape: {df.shape}")
print(f"📋 Columns ({len(df.columns)}): {list(df.columns)}")

# Display basic statistics
print(f"\n📈 Quick Statistics:")
print(f"  Total records: {len(df):,}")
print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# Inspect columns and identify key fields - Enhanced with Data Lineage
from datetime import datetime
print("📋 Available columns:")
for i, col in enumerate(df.columns, 1):
    non_null_count = df[col].count()
    null_pct = ((len(df) - non_null_count) / len(df) * 100) if len(df) > 0 else 0
    print(f"  {i:2d}. {col:<30} ({non_null_count:,} non-null, {null_pct:.1f}% null)")

# Enhanced key columns tracking with metadata
key_columns = {
    'record_id': None,
    'match_key': None,
    'borrowdir_results': None,
    'hsp_filtered': False,
    'processing_date': datetime.now().strftime("%Y-%m-%d"),
    'source_file': loaded_from,
    'data_lineage': []
}

# Identify record ID column with validation
for col_name in ['F001', 'record_id', 'mms_id', 'MMSID']:
    if col_name in df.columns:
        key_columns['record_id'] = col_name
        key_columns['data_lineage'].append(f"Using {col_name} as record identifier")
        break

# Identify match key column with validation
for col_name in ['unique_match_key', 'match_key', 'normalized_match_key']:
    if col_name in df.columns:
        key_columns['match_key'] = col_name
        key_columns['data_lineage'].append(f"Using {col_name} for match key comparison")
        break

# Check for existing BorrowDirect results with validation
for col_name in ['borrowdir_ids', 'borrowdir_id', 'borrowdirect_ids', 'borrowdirect_results']:
    if col_name in df.columns:
        key_columns['borrowdir_results'] = col_name
        key_columns['data_lineage'].append(f"Found existing BorrowDirect results in {col_name}")
        break

# Enhanced HSP filtering detection
hsp_status = {
    'filtered': False,
    'source': None,
    'date': None
}

if loaded_from:
    # Check filename for HSP indicators
    if any(term in loaded_from.lower() for term in ['hsp', 'no_hsp', 'filtered']):
        hsp_status['filtered'] = True
        hsp_status['source'] = 'filename'
        key_columns['data_lineage'].append(f"HSP filtering detected from filename: {loaded_from}")

# Check for explicit HSP filtering columns
if 'hsp_filtered' in df.columns:
    hsp_status['filtered'] = True
    hsp_status['source'] = 'column'
    key_columns['data_lineage'].append("HSP filtering verified through column presence")

# Check for HSP filtering timestamp if available
if 'hsp_filtered_date' in df.columns:
    hsp_status['date'] = df['hsp_filtered_date'].iloc[0]
    key_columns['data_lineage'].append(f"HSP filtering date found: {hsp_status['date']}")

key_columns['hsp_filtered'] = hsp_status['filtered']

# Print enhanced status report
print(f"\n=== Data Processing Status ===")
print(f"🔄 Processing Date: {key_columns['processing_date']}")
print(f"📂 Source File: {key_columns['source_file']}")

print(f"\n🔑 Key Columns Status:")
for key, value in key_columns.items():
    if key not in ['processing_date', 'source_file', 'data_lineage']:
        status = "✅" if value else ("⚠️" if key == 'borrowdir_results' else "❌")
        print(f"  {status} {key}: {value}")

print(f"\n📋 Data Lineage:")
for step in key_columns['data_lineage']:
    print(f"  • {step}")

# Enhanced institution analysis
institution_cols = [col for col in df.columns if 'institution' in col.lower() or col in ['POD_organization']]
if institution_cols:
    print(f"\n🏛️ Institution Columns:")
    for col in institution_cols:
        unique_values = df[col].nunique()
        print(f"  • {col} ({unique_values:,} unique values)")

# Enhanced data sample display
print(f"\n📊 Data Sample (first 3 rows):")
display_cols = []
if key_columns['record_id']:
    display_cols.append(key_columns['record_id'])
if key_columns['match_key']:
    display_cols.append(key_columns['match_key'])
if key_columns['borrowdir_results']:
    display_cols.append(key_columns['borrowdir_results'])
if institution_cols:
    display_cols.extend(institution_cols[:1])

# Add key fields for analysis
for field in ['F245', 'F020']:  # Title and ISBN fields
    if field in df.columns:
        display_cols.append(field)

if display_cols:
    print(df[display_cols].head(3))
else:
    print(df.head(3))

# Save processing metadata
processing_metadata = {
    'processing_date': key_columns['processing_date'],
    'source_file': key_columns['source_file'],
    'data_lineage': key_columns['data_lineage'],
    'hsp_status': hsp_status
}

# Store metadata in DataFrame
df.attrs['processing_metadata'] = processing_metadata

Index(['key', 'F001', 'F010_str', 'F245', 'normalized_title',
       'normalized_edition', 'normalized_pub', 'source', 'match_key',
       'F007_str', 'F020_str', 'F250_str', 'F260_str', 'id_list_str',
       'key_array_str', 'F007_code', 'F007_desc'],
      dtype='object')


In [ ]:
# Format record ID if needed - Enhanced
if key_columns['record_id']:
    record_col = key_columns['record_id']
    print(f"🔧 Formatting {record_col} column...")
    
    # Store original type for comparison
    original_dtype = df[record_col].dtype
    original_sample = df[record_col].head().tolist()
    
    # Ensure record ID is a string, then apply specific transformations
    df[record_col] = df[record_col].astype(str)
    
    # Replace any occurrence ending with "03680" with "03681" (known data correction)
    corrections_made = df[record_col].str.contains(r'03680$', regex=True, na=False).sum()
    if corrections_made > 0:
        df[record_col] = df[record_col].str.replace(r'03680$', '03681', regex=True)
        print(f"  ✅ Applied {corrections_made} record ID corrections (03680 → 03681)")
    
    # Remove any 'nan' strings that might have been created
    nan_count = (df[record_col] == 'nan').sum()
    if nan_count > 0:
        df[record_col] = df[record_col].replace('nan', pd.NA)
        print(f"  ✅ Cleaned {nan_count} 'nan' string values")
    
    print(f"  Original dtype: {original_dtype}")
    print(f"  New dtype: {df[record_col].dtype}")
    print(f"  Sample original values: {original_sample}")
    print(f"  Sample formatted values: {df[record_col].head().tolist()}")
    
    # Check for any remaining issues
    null_count = df[record_col].isnull().sum()
    if null_count > 0:
        print(f"  ⚠️ Warning: {null_count} null values in record ID column")
else:
    print("⚠️ No record ID column found - skipping record ID formatting")
    print("Available columns:", list(df.columns))

                                                 key              F001  \
0  8604 forrest avenue philadelphia pennsylvania ...  9978845258603681   
1                                      9789381005408  9978085185803681   
2                                      9788170565628  9977914437003681   
3                            9788126423415 paperback  9962328533503681   
4                                         8192611396  9978003905503681   

  F010_str                                               F245  \
0      NaN  8604 Forrest avenue, Philadelphia, Pennsylvani...   
1      NaN  880-01 Bhāratīya citrakalā meṃ Jaina citra...   
2      NaN  880-01 Kamaleśvara ke kathā-sāhitya meṃ ma...   
3      NaN  880-01 Mālguḍidinaṅṅaḷ / Ār. Ke. Nārāy...   
4      NaN  880-01 Mōhanasvāmi : kathāsaṅkalana / Vasu...   

                                    normalized_title       normalized_edition  \
0  8604 forrest avenue philadelphia pennsylvania ...                      NaN   
1 

In [ ]:
# Check match key uniqueness and completeness
if key_columns['match_key']:
    match_col = key_columns['match_key']
    print(f"🔍 Analyzing {match_col} column...")
    
    # Basic statistics
    total_records = len(df)
    unique_keys = df[match_col].nunique()
    is_unique = df[match_col].is_unique
    
    print(f"  📊 Basic Statistics:")
    print(f"    Total records: {total_records:,}")
    print(f"    Unique match keys: {unique_keys:,}")
    print(f"    All keys unique: {is_unique}")
    if not is_unique:
        duplicates = total_records - unique_keys
        print(f"    Duplicate records: {duplicates:,} ({duplicates/total_records*100:.1f}%)")
    
    # Check for null/empty values
    null_count = df[match_col].isnull().sum()
    empty_count = (df[match_col] == '').sum() if df[match_col].dtype == 'object' else 0
    total_missing = null_count + empty_count
    
    print(f"  🚫 Missing Data:")
    print(f"    Null values: {null_count:,}")
    print(f"    Empty strings: {empty_count:,}")
    print(f"    Total missing: {total_missing:,} ({total_missing/total_records*100:.1f}%)")
    
    # Analyze match key patterns
    if df[match_col].dtype == 'object' and total_missing < total_records:
        valid_keys = df[match_col].dropna()
        valid_keys = valid_keys[valid_keys != '']
        
        if len(valid_keys) > 0:
            key_lengths = valid_keys.str.len()
            print(f"  📏 Key Length Analysis:")
            print(f"    Min length: {key_lengths.min()}")
            print(f"    Max length: {key_lengths.max()}")
            print(f"    Median length: {key_lengths.median()}")
            
            # Show sample keys of different lengths
            print(f"  🔤 Sample keys:")
            for i, key in enumerate(valid_keys.head(5)):
                print(f"    {i+1}. {key[:50]}{'...' if len(key) > 50 else ''} (len: {len(key)})")
    
    # If there are missing match keys, show sample records
    if total_missing > 0:
        print(f"\n⚠️ Records with missing match keys:")
        missing_sample = df[df[match_col].isnull() | (df[match_col] == '')]
        display_cols = [key_columns['record_id'], match_col]
        display_cols = [col for col in display_cols if col is not None]
        
        # Add some additional identifying columns if available
        for col in ['F245', 'title', 'F020', 'isbn', 'POD_organization']:
            if col in df.columns:
                display_cols.append(col)
                break
        
        print(missing_sample[display_cols].head())
        
        # Option to filter out missing keys
        print(f"\n❓ Should we filter out records with missing match keys? ({total_missing:,} would be removed)")
else:
    print("❌ No match key column found - cannot proceed with BorrowDirect verification")
    print("This is required for API calls. Please check the main pipeline outputs.")

True


In [ ]:
# HSP filtering - Conditional and Enhanced
initial_count = len(df)

if key_columns['hsp_filtered']:
    print("✅ HSP filtering already applied in main pipeline - skipping")
    print(f"   Current record count: {initial_count:,}")
    
elif os.path.exists('hsp/hsp-removed-mmsid.txt'):
    print("🔧 Applying HSP filtering from hsp/hsp-removed-mmsid.txt...")
    
    # Load HSP MMSIDs to remove
    with open('hsp/hsp-removed-mmsid.txt') as f:
        hsp_removed_mmsid = [line.strip() for line in f.readlines() if line.strip()]
    
    print(f"   Loaded {len(hsp_removed_mmsid):,} HSP MMSIDs to remove")
    
    # Get the record ID column
    record_col = key_columns['record_id']
    if record_col:
        # Apply HSP filtering: Remove rows with MMSIDs that are in hsp_removed_mmsid
        before_count = len(df)
        
        # Convert both to strings for comparison
        df[record_col] = df[record_col].astype(str)
        hsp_set = set(str(mmsid) for mmsid in hsp_removed_mmsid)
        
        # CORRECT: Remove rows with MMSIDs that are in hsp_removed_mmsid (use ~ for NOT)
        mask = ~df[record_col].isin(hsp_set)
        df = df[mask].copy()
        
        after_count = len(df)
        removed_count = before_count - after_count
        
        print(f"✅ HSP filtering complete:")
        print(f"   Records before: {before_count:,}")
        print(f"   Records after: {after_count:,}")
        print(f"   Records removed: {removed_count:,} ({removed_count/before_count*100:.1f}%)")
        
        if removed_count == 0:
            print("   ℹ️ No records were removed - HSP MMSIDs may not be present in this dataset")
    else:
        print("❌ Cannot apply HSP filtering - no record ID column found")
        
elif os.path.exists('hsp-removed-mmsid.txt'):
    print("🔧 Applying HSP filtering from current directory...")
    
    with open('hsp-removed-mmsid.txt') as f:
        hsp_removed_mmsid = [line.strip() for line in f.readlines() if line.strip()]
    
    record_col = key_columns['record_id'] 
    if record_col:
        before_count = len(df)
        df[record_col] = df[record_col].astype(str)
        hsp_set = set(str(mmsid) for mmsid in hsp_removed_mmsid)
        mask = ~df[record_col].isin(hsp_set)  # CORRECT: Added ~ operator
        df = df[mask].copy()
        
        after_count = len(df)
        removed_count = before_count - after_count
        
        print(f"✅ HSP filtering complete: removed {removed_count:,} records")
    else:
        print("❌ Cannot apply HSP filtering - no record ID column found")
        
else:
    print("⚠️ HSP filtering file not found - proceeding without HSP filtering")
    print("   This may be acceptable if HSP filtering was already applied in the main pipeline")

print(f"\n📊 Current dataset size: {len(df):,} records")

In [ ]:
# BorrowDirect API Integration - Enhanced and Conditional
import time
import requests
import pandas as pd
from typing import List, Union

def get_borrowdir_ids(match_key: str, max_retries: int = 3) -> List[str]:
    """
    Fetch BorrowDirect IDs for a given match key with error handling and retries.
    """
    if pd.isna(match_key) or match_key == '':
        return []
    
    url = f"https://borrowdirect.reshare.indexdata.com/api/v1/search?lookfor={match_key}"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raise an exception for bad status codes
            
            data = response.json()
            # Collect all ids; use set() to ensure uniqueness
            ids = list(set(record['id'] for record in data.get('records', [])))
            time.sleep(1)  # Throttle the requests
            return ids
            
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                print(f"⚠️ API error for {match_key} (attempt {attempt + 1}): {e}")
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                print(f"❌ Failed to fetch BorrowDirect IDs for {match_key}: {e}")
                return []
        except Exception as e:
            print(f"❌ Unexpected error for {match_key}: {e}")
            return []

# Check if BorrowDirect results already exist
if key_columns['borrowdir_results']:
    borrowdir_col = key_columns['borrowdir_results']
    print(f"✅ Existing BorrowDirect results found in column: {borrowdir_col}")
    
    # Check if results are populated
    non_empty_results = df[borrowdir_col].notna().sum()
    total_results = len(df)
    
    print(f"   Records with BorrowDirect data: {non_empty_results:,} / {total_results:,} ({non_empty_results/total_results*100:.1f}%)")
    
    # Sample some results
    sample_results = df[df[borrowdir_col].notna()][borrowdir_col].head()
    print(f"   Sample results: {sample_results.tolist()}")
    
    # Ask if we should use existing results or fetch fresh ones
    use_existing = True  # Default to using existing results
    print(f"   ✅ Using existing BorrowDirect results")
    
else:
    use_existing = False
    print("🔄 No existing BorrowDirect results found - will fetch from API")

# Fetch BorrowDirect data if needed
if not use_existing and key_columns['match_key']:
    match_col = key_columns['match_key']
    
    # Filter out records with missing match keys
    valid_df = df[df[match_col].notna() & (df[match_col] != '')].copy()
    
    print(f"🔄 Fetching BorrowDirect IDs for {len(valid_df):,} records...")
    print(f"   This may take {len(valid_df) * 1.2 / 60:.1f} minutes with API throttling")
    
    # Apply the function to get BorrowDirect IDs
    valid_df['borrowdir_ids'] = valid_df[match_col].apply(get_borrowdir_ids)
    
    # Merge back to original dataframe
    df = df.merge(valid_df[['borrowdir_ids']], left_index=True, right_index=True, how='left')
    df['borrowdir_ids'] = df['borrowdir_ids'].fillna([])  # Fill NaN with empty lists
    
    key_columns['borrowdir_results'] = 'borrowdir_ids'
    print(f"✅ BorrowDirect data fetching complete")
    
    # Show summary
    has_results = df['borrowdir_ids'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False).sum()
    print(f"   Records with BorrowDirect matches: {has_results:,} / {len(df):,} ({has_results/len(df)*100:.1f}%)")

elif not key_columns['match_key']:
    print("❌ Cannot fetch BorrowDirect data - no match key column available")

print(f"\n📊 Current dataset status:")
print(f"   Total records: {len(df):,}")
if key_columns['borrowdir_results']:
    borrowdir_col = key_columns['borrowdir_results']
    has_borrowdir = df[borrowdir_col].notna().sum()
    print(f"   Records with BorrowDirect data: {has_borrowdir:,}")

                       key              F001 F010_str  \
1            9789381005408  9978085185803681      NaN   
2            9788170565628  9977914437003681      NaN   
3  9788126423415 paperback  9962328533503681      NaN   
4               8192611396  9978003905503681      NaN   
5            9788126435432  9959112523503681      NaN   

                                                F245  \
1  880-01 Bhāratīya citrakalā meṃ Jaina citra...   
2  880-01 Kamaleśvara ke kathā-sāhitya meṃ ma...   
3  880-01 Mālguḍidinaṅṅaḷ / Ār. Ke. Nārāy...   
4  880-01 Mōhanasvāmi : kathāsaṅkalana / Vasu...   
5  880-01 Prācīna lōkacaritraṃ / Her̲oḍōṭt...   

                                    normalized_title       normalized_edition  \
1  880-01 bhāratīya citrakalā meṃ jaina citra...  880-02 1. saṃskaraṇa.   
2  880-01 kamaleśvara ke kathā-sāhitya meṃ ma...   880-02 saṃskaraṇa 1.   
3  880-01 mālguḍidinaṅṅaḷ / ār. ke. nārāy...                      NaN

In [ ]:
# Save intermediate results to standardized output directory
import os

# Ensure output directory exists
os.makedirs('pod-processing-outputs', exist_ok=True)

# Save current state with BorrowDirect results
output_file = 'pod-processing-outputs/post-processing-with-borrowdir_ids.csv'
df.to_csv(output_file, index=False)

print(f"✅ Saved {len(df):,} records to {output_file}")

# Also save as Parquet for better performance
parquet_file = 'pod-processing-outputs/post-processing-with-borrowdir_ids.parquet'
df.to_parquet(parquet_file, index=False)

print(f"✅ Saved {len(df):,} records to {parquet_file}")

# Display summary statistics
print(f"\n📊 Saved Dataset Summary:")
print(f"   Records: {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print(f"   File size (CSV): {os.path.getsize(output_file) / 1024**2:.1f} MB")
print(f"   File size (Parquet): {os.path.getsize(parquet_file) / 1024**2:.1f} MB")

In [46]:
!pip install selenium


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import time
import math
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def check_up_holdings_selenium(borrowdir_id: str, debug: bool = False) -> bool:
    """
    Check if holdings are exclusive to University of Pennsylvania using Selenium.
    Returns True if only Penn holds the item, False otherwise.
    """
    # Skip if borrowdir_id is None, NaN, or empty
    if (borrowdir_id is None or 
        (isinstance(borrowdir_id, float) and math.isnan(borrowdir_id)) or
        borrowdir_id == '' or borrowdir_id == 'nan'):
        if debug:
            print("Skipping due to empty/invalid borrowdir_id")
        return False

    url = f"https://borrowdirect.reshare.indexdata.com/Record/{borrowdir_id}/Holdings"

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    
    driver = None
    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.set_page_load_timeout(30)
        
        if debug:
            print(f"Accessing URL: {url}")
        
        driver.get(url)
        
        # Wait for dynamic content to load
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.tab-content")))
        
        # Locate the main tab content container
        tab_content = driver.find_element(By.CSS_SELECTOR, "div.tab-content")
        
        # Within tab_content, get the active holdings pane
        holdings_div = tab_content.find_element(By.CSS_SELECTOR, "div.tab-pane.holdings-tab.active")
        
        # Look for h3 elements within the holdings pane
        h3_tags = holdings_div.find_elements(By.TAG_NAME, "h3")
        institutions = set(tag.text.strip() for tag in h3_tags if tag.text.strip())
        
        if debug:
            print(f"Institutions found: {institutions}")
        
        # Check if only University of Pennsylvania holds the item
        result = (institutions == {"University of Pennsylvania"})
        
        if debug and result:
            print("✅ Penn-only holding confirmed")
        elif debug:
            print(f"❌ Multiple institutions: {institutions}")
            
        return result
        
    except Exception as e:
        if debug:
            print(f"Error encountered: {e}")
        return False
    finally:
        if driver:
            driver.quit()

# Example debugging usage
print(check_up_holdings_selenium("YOUR_KNOWN_BORROWDIR_ID", debug=True))

# Prepare data for holdings verification
if key_columns['borrowdir_results']:
    borrowdir_col = key_columns['borrowdir_results']
    
    print(f"🔍 Preparing holdings verification using {borrowdir_col} column...")
    
    # Create a copy for processing
    verification_df = df.copy()
    
    # Explode borrowdir_ids if they are in list format
    if verification_df[borrowdir_col].apply(lambda x: isinstance(x, list)).any():
        print("   📊 Exploding list-format BorrowDirect IDs...")
        verification_df = verification_df.explode(borrowdir_col).reset_index(drop=True)
        print(f"   Expanded to {len(verification_df):,} records for verification")
    
    # Filter out empty/invalid BorrowDirect IDs
    valid_mask = (
        verification_df[borrowdir_col].notna() & 
        (verification_df[borrowdir_col] != '') & 
        (verification_df[borrowdir_col] != 'nan')
    )
    
    if verification_df[borrowdir_col].dtype == 'object':
        # Handle list columns that might have been converted to strings
        valid_mask = valid_mask & ~verification_df[borrowdir_col].str.startswith('[')
    
    verification_df = verification_df[valid_mask].copy()
    
    print(f"   📊 Records ready for verification: {len(verification_df):,}")
    
    if len(verification_df) > 0:
        # Sample a few records for testing first
        sample_size = min(5, len(verification_df))
        sample_df = verification_df.head(sample_size).copy()
        
        print(f"   🧪 Testing with {sample_size} sample records first...")
        
        # Test with debug output
        sample_df['up_holdings'] = sample_df[borrowdir_col].apply(
            lambda x: check_up_holdings_selenium(x, debug=True)
        )
        
        # Show results
        penn_only_count = sample_df['up_holdings'].sum()
        print(f"   📊 Sample results: {penn_only_count}/{sample_size} are Penn-only holdings")
        
        if penn_only_count > 0:
            print(f"   ✅ Found Penn-only holdings - proceeding with full verification")
            
            # Apply to full dataset (without debug for speed)
            print(f"   🔄 Verifying all {len(verification_df):,} records...")
            verification_df['up_holdings'] = verification_df[borrowdir_col].apply(
                lambda x: check_up_holdings_selenium(x, debug=False)
            )
            
            # Summary statistics
            total_penn_only = verification_df['up_holdings'].sum()
            print(f"   ✅ Holdings verification complete!")
            print(f"     Total verified records: {len(verification_df):,}")
            print(f"     Penn-only holdings: {total_penn_only:,} ({total_penn_only/len(verification_df)*100:.1f}%)")
            
        else:
            print(f"   ⚠️ No Penn-only holdings found in sample - check BorrowDirect data quality")
            verification_df['up_holdings'] = False
    else:
        print("   ❌ No valid BorrowDirect IDs found for verification")
        verification_df['up_holdings'] = False
        
else:
    print("❌ No BorrowDirect results column found - cannot perform holdings verification")
    verification_df = df.copy()
    verification_df['up_holdings'] = False

Accessing URL: https://borrowdirect.reshare.indexdata.com/Record/YOUR_KNOWN_BORROWDIR_ID/Holdings
Error encountered: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.tab-content"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000101102138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x00000001010fa988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x0000000100b60968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000100ba4d4c cxxbridge1$string$len + 368752
4   chromedriver                        0x0000000100bde4f0 cxxbridge1$string$len + 604180
5   chromedriver                        0x0000000100b99564 cxxbridge1$string$len + 321672
6   chromedriver                        0x0

In [ ]:
import pandas as pd
import os

# Ensure output directory exists
os.makedirs('pod-processing-outputs', exist_ok=True)

# Filter records where only Penn Libraries are the holding institution
if 'up_holdings' in verification_df.columns:
    df_penn_only = verification_df[verification_df['up_holdings'] == True].copy()
    
    # Remove duplicates based on match key or record ID
    if key_columns['match_key']:
        initial_count = len(df_penn_only)
        df_penn_only = df_penn_only.drop_duplicates(subset=[key_columns['match_key']])
        dedup_count = initial_count - len(df_penn_only)
        if dedup_count > 0:
            print(f"🔧 Removed {dedup_count:,} duplicate records based on match key")
    elif key_columns['record_id']:
        initial_count = len(df_penn_only)
        df_penn_only = df_penn_only.drop_duplicates(subset=[key_columns['record_id']])
        dedup_count = initial_count - len(df_penn_only)
        if dedup_count > 0:
            print(f"🔧 Removed {dedup_count:,} duplicate records based on record ID")
    
    print(f"\n📊 Final Results Summary:")
    print(f"   Total records processed: {len(df):,}")
    print(f"   Records with BorrowDirect data: {len(verification_df):,}")
    print(f"   Penn-only holdings found: {len(df_penn_only):,}")
    print(f"   Percentage unique to Penn: {len(df_penn_only)/len(verification_df)*100:.1f}%")
    
    if len(df_penn_only) > 0:
        # Export to Excel
        excel_output = "pod-processing-outputs/penn_unique_confirmed.xlsx"
        df_penn_only.to_excel(excel_output, index=False)
        print(f"✅ Exported {len(df_penn_only):,} Penn-only records to {excel_output}")
        
        # Also save as Parquet
        parquet_output = "pod-processing-outputs/penn_unique_confirmed.parquet"
        df_penn_only.to_parquet(parquet_output, index=False)
        print(f"✅ Exported {len(df_penn_only):,} Penn-only records to {parquet_output}")
        
        # Save detailed verification results
        verification_output = "pod-processing-outputs/holdings_verification_results.parquet"
        verification_df.to_parquet(verification_output, index=False)
        print(f"✅ Saved full verification results to {verification_output}")
        
        # Display sample of Penn-only records
        print(f"\n📋 Sample Penn-only holdings:")
        display_cols = []
        if key_columns['record_id']:
            display_cols.append(key_columns['record_id'])
        if key_columns['match_key']:
            display_cols.append(key_columns['match_key'])
        
        # Add title/isbn columns if available
        for col in ['F245', 'title', 'F020', 'isbn']:
            if col in df_penn_only.columns:
                display_cols.append(col)
                break
        
        print(df_penn_only[display_cols].head() if display_cols else df_penn_only.head())
        
        # Create summary statistics
        summary = {
            'total_records_processed': len(df),
            'records_with_borrowdir_data': len(verification_df),
            'penn_only_holdings': len(df_penn_only),
            'percentage_unique_to_penn': round(len(df_penn_only)/len(verification_df)*100, 2),
            'input_file': loaded_from,
            'output_files': [excel_output, parquet_output, verification_output]
        }
        
        # Save summary
        summary_output = "pod-processing-outputs/final_verification_summary.json"
        import json
        with open(summary_output, 'w') as f:
            json.dump(summary, f, indent=2)
        print(f"✅ Saved processing summary to {summary_output}")
        
    else:
        print("⚠️ No Penn-only holdings found - check data quality and verification logic")
        
else:
    print("❌ No holdings verification was performed - cannot create Penn-only export")
    print("Please ensure the holdings verification step completed successfully.")

Exported Excel file with Penn Only holdings to penn_only_holdings.xlsx


# HathiTrust Digital Availability Check


In [ ]:

# Check which unique Penn holdings are already digitized in HathiTrust
print("\n" + "="*60)
print("HATHITRUST DIGITAL AVAILABILITY CHECK")
print("="*60)

# Import required modules
import sys
import os
import pandas as pd

# Add HathiTrust directory to path
sys.path.append('hathitrust')

try:
    from hathitrust_availability_checker_excel import HathiTrustFullScanner
    
    # Use the verified Penn-only holdings
    if 'df_penn_only' in locals() and len(df_penn_only) > 0:
        print(f"\nChecking {len(df_penn_only):,} Penn-only holdings for HathiTrust availability...")
        
        # Save temporary Excel file with proper column names
        temp_file = 'pod-processing-outputs/temp_hathitrust_input.xlsx'
        
        # Prepare columns for HathiTrust checker
        hathi_df = pd.DataFrame({
            'MMS_ID': df_penn_only['F001'] if 'F001' in df_penn_only.columns else df_penn_only.index,
            'F245': df_penn_only['F245'] if 'F245' in df_penn_only.columns else '',
            'F020_str': df_penn_only['F020'].astype(str) if 'F020' in df_penn_only.columns else '',
            'F010_str': df_penn_only['F010'].astype(str) if 'F010' in df_penn_only.columns else '',
            'F260_str': df_penn_only['F260'].astype(str) if 'F260' in df_penn_only.columns else '',
            'id_list_str': df_penn_only['F035'].astype(str) if 'F035' in df_penn_only.columns else '',
            'borrowdir_id': df_penn_only[borrowdir_col] if borrowdir_col in df_penn_only.columns else ''
        })
        
        # Save to Excel
        hathi_df.to_excel(temp_file, index=False)
        print(f"✅ Prepared data saved to: {temp_file}")
        
        # Initialize scanner with conservative rate limiting
        scanner = HathiTrustFullScanner(rate_limit_delay=0.3, max_workers=3)
        
        # Run the scan
        print("\nStarting HathiTrust scan...")
        print("This may take several minutes depending on the number of records...")
        scanner.scan_full_file(temp_file, batch_size=50)
        
        # Results are automatically saved by the scanner
        print("\n✅ HathiTrust check complete!")
        print("Check the 'hathitrust/reports' directory for detailed results")
        
        # Clean up temporary file
        if os.path.exists(temp_file):
            os.remove(temp_file)
            
    else:
        print("❌ No Penn-only holdings found to check")
        print("Please ensure the holdings verification step completed successfully")
        
except ImportError:
    print("❌ Could not import HathiTrust scanner")
    print("Please ensure hathitrust-availability-checker-excel.py is in the hathitrust/ directory")
except Exception as e:
    print(f"❌ Error during HathiTrust check: {str(e)}")
    
    # Clean up on error
    if 'temp_file' in locals() and os.path.exists(temp_file):
        os.remove(temp_file)

## Post-Processing Complete ✅

This notebook has successfully completed the post-processing verification workflow:

### Key Accomplishments:
1. **✅ Data Loading**: Automatically detected and loaded the most appropriate dataset from main pipeline outputs
2. **✅ Column Mapping**: Dynamically identified record IDs, match keys, and BorrowDirect results columns
3. **✅ HSP Filtering**: Applied conditional HSP filtering only when needed
4. **✅ API Integration**: Leveraged existing BorrowDirect results or fetched fresh data with robust error handling
5. **✅ Holdings Verification**: Used Selenium to verify Penn-exclusive holdings
6. **✅ Export & Documentation**: Saved results to standardized output directory with comprehensive summary

### Output Files:
- `pod-processing-outputs/penn_unique_confirmed.xlsx` - Final Penn-only holdings (Excel format)
- `pod-processing-outputs/penn_unique_confirmed.parquet` - Final Penn-only holdings (Parquet format)
- `pod-processing-outputs/holdings_verification_results.parquet` - Complete verification results
- `pod-processing-outputs/final_verification_summary.json` - Processing summary statistics

### Next Steps:
- Review the exported Penn-only holdings for manual validation
- Use the verification results for statistical analysis
- Consider the processing summary for pipeline optimization

The post-processing pipeline is now fully aligned with the main processing pipeline and provides robust verification of Penn's unique holdings.